In [1]:
# libraries
import numpy as np
import pandas as pd
import pickle as pkl
import sys
sys.path.append("../functions/")
from Boostrap import Bootstrap, regress

import warnings
warnings.filterwarnings("ignore")

# first and last Monday
from datetime import datetime, timedelta
start_date, end_date  = datetime(2020, 3, 2), datetime(2020, 12, 21)
window = timedelta(days=7)
dates = pd.to_datetime(np.arange(start_date, end_date + window, window))

def time_regress(country, ycol, features):
    """
    This function runs the regression in time
    :param df: standardized df
    :param ycol: name of y col
    :param features: name of independent features
    :return: dictionary of regression results
    """
    results, results_nombps = {}, {}

    # import data
    df = pd.read_csv("./input-dfs-time/" + country + "_time_std.csv")
    for week in range(len(dates)):
        # loc data in this week
        df_week = df.loc[df.week == week]
        # regress
        results[week] = regress(df_week, features, ycol)
        # regress without mbps
        results_nombps[week] = regress(df_week, [f for f in features if f != 'download_mbps_fixed'], ycol)

    return results, results_nombps

In [2]:
for country in ['colombia', 'ecuador', 'el-salvador']:
    print("\n" + country)
    for ycol in ["max_movs", "max_stay"]:
        print("\t" + ycol)

        if country in ["el-salvador", "ecuador"]:
            features = ["download_mbps_fixed", "rwi_weight", "gdp_per_capita", "pop2020", "pop_density", "pop60plus_ratio"]
        elif country == "colombia":
            features = ["download_mbps_fixed", "rwi_weight", "gdp_per_capita", "popDANE", "pop_density", "pop60plus_ratio"]

        regressions = {}
        regressions['complete'], regressions['complete_nombps'] = time_regress(country, ycol, features)
        with open("./output-time/" + country + "_" + ycol + "_time.pkl", "wb") as file:
            pkl.dump(regressions, file)


colombia
	max_movs
	max_stay

ecuador
	max_movs
	max_stay

el-salvador
	max_movs
	max_stay
